<h2 style="text-align:center">Makine Öğrenme Yöntemleriyle Sahte Kullanıcı Yorumlarının Tespiti</h2>

<img src="https://i.hizliresim.com/b7h62o.jpg">

<h3>Proje İçeriği</h3>

<ul>

<li>1- Giriş</li>
<li>2- Data Setinin Oluşturulması</li>
<li>3- Proje İçin Kullanılan Yöntemler</li>
<li>3.1- LSTM</li>
<li>3.2- Lojistik Regrasyon</li>
<li>4- Kullanılan Algoritmaların Karşılaştırılması (Sonuç)</li>

</ul>

<h2>1- Giriş</h2>

E-ticaret sistemi çok geliştiği için ürünler hakkında birçok yorum yapılmaktadır. Yapay zekanın gelişmesiyle birlikte verilerin gerçekliği hakkında şüpheler başlamıştır. Bu projede bu soruna çözüm aranmaktadır. Geliştirilen sistem şu şekilde çalışmaktadır. Öncelikle klavyeden bir giriş alınır. Daha sonra eğitilmiş modelimiz bunun makine yorumu mu (sahte yorum) yoksa insan yorumu mu olduğunu tespit etmektedir.


<h2>2- Veri Setinin Oluşturulması</h2>

Data setinin türkçe olması için datasetini kendimiz oluşturduk. Datasetinde toplamda 2549 adet yorum bulunmaktadır. Bu verilerin <b>1200</b> tanesi gerçek kullanıcı yorumudur. Geriye kalan veriler fake olarak üretilmiştir. Fake yorumlar ise GRU (Lstm nöron çeşiti) ile geliştirilmiştir. https://github.com/Aksoylu adresindeki bir proje kaynak alınarak 1349 adet fake veri üretilmiştir.

Doğal yorumlar hepsiburada'da yer alan ürün yorumlarından toplanmıştır. Genellikle maskara, ayakkabı, çanta, telefon, kulaklık gibi ürünlerin yorumlarından oluşmuştur. Datasetimizi oluştururken <b>Kullanıcı yorumları 1</b> ile ifade edilirken <b>fake yorumlar 0</b> ile gösterilmiştir. Bunun için oluşturulan datasetinde <b>yorum</b> ve <b>etiket</b> sütunları bulunmaktadır.

In [1]:
pip install beautifulsoup4

You should consider upgrading via the 'c:\users\melike\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd

In [2]:
#https://www.hepsiburada.com/maybelline-new-york-lash-sensational-sky-high-maskara-p-HBV00001B1F33-yorumlari?magaza=AtakanKozmetika&sayfa=1
from bs4 import BeautifulSoup
import requests


In [26]:
link = "https://www.hepsiburada.com/lenovo-ideapad-5-amd-ryzen-7-5700u-8gb-512gb-ssd-freedos-14-fhd-tasinabilir-bilgisayar-82lm006etx-p-HBCV00000471GC-yorumlari?sayfa=1"
headersparam = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"}
r = requests.get(link, headers=headersparam)
soup = BeautifulSoup(r.content, "html", from_encoding='UTF-8')
span = soup.find_all("span",{"itemprop": "description"})

for i in span:
    print(i.text)

Ürün gerçekten güzel ve kalite hissini veriyor.
Ekran çok güzel ıps olduğundan ama parlaklık biraz daha iyi olabilirdi dışarıda güneşte en son parlaklıkta kullanmak gerekecek.
Hafıza gerçekten yeterli.
Oyun oynatıyor çok aşırı olmadıkça yani gta 5 veya csgo çok rahat oynanıyor işlemcisi buna çok müsait.
Açılıp kapanma hızı gerçekten çok güzel.
Hafıza yeterli.
Ram hızı 3200 bilgisayarlara bakarken buna da bakın arkadaşlar genelde ram hızı 2600 olur bunda 3200 bu güzel bir avantaj.
Ram değişmiyor bilginiz olsun.
Klavye ortalama üstü ama çok iyi değil.
Malzeme kalitesi olarak güzel kullanırken sıradan bir ürün olmadığını gösteriyor.
Genel olarak fiyatının hakkını sonuna kadar veriyor bu fiyata daha iyisi yok. Alacaklara hayırlı olsun.
tek eksik hissettiğim belki klavye ışığı olabilirdi. gece çalışırken rahat olurdu. laptop süper , tam bir kompakt , hızlı , heryere taşınabilir. 4k görüntü veriyor hdmı ile tv ye. tavsiye ederim.
Öncelikle kargo çok hızlı geldiğini belirtmek istiyorum. Son s

In [3]:
data = pd.read_csv('dataset.csv') #Oluşturulan verisetinin gösterilmesi
print("Verisetinin Boyutu: ")
print(data.shape)
data.head(1000)

Verisetinin Boyutu: 
(2549, 3)


Unnamed: 0                                              Yorum  Etiket
0             0  maskara ürünü desteklemeyen yerlir görmedim yi...       0
1             1  maskarafı öne kullanıyorum yada fark atıyor\r\...       0
2             2  maskarada bulunan dokunma diğeri  kere iti akt...       0
3             3  maskara girmesini beklemeyin üst seviye\r\ngüz...       0
4             4  maskara kıyasla uygun fiyatlı olması kolay ay ...       0
..          ...                                                ...     ...
995         995  mousekablo derdinden kurtuldum henüz yukarı ka...       0
996         996  mousekablo uyumu vardır amcam tane almıştım ha...       0
997         997  mouse bişey imi kaçırmasın\r\ngüzel\r\nsadece ...       0
998         998  mouseleri ergonomik bir telefon şimdiden almam...       0
999         999  mouse vb hediye haksız biraz korkularım yanınd...       0

[1000 rows x 3 columns]

<h2>3- Proje İçin Kullanılan Algoritmalar</h2>

Bu projede iki adet algoritma kullanılmıştır. Bunlardan bir tanesi <b>LSTM</b> diğeri ise <b>Lojistik Regrasyondur</b>. İki adet algoritma kullanılma sebebi ise hangi algoritmanın daha iyi sonuç vereceğini tespit etmektir.
Bu algoritmaların karşılaştırılması ve sonuçları ilgili kısımlarda anlatılacaktır.


LSTM yapısı itibariyle bir model eğitilirken zaman almaktadır. Bu algoritmayı tercih etmemizin sebebi ise daha iyi öğrenme gerçekleşeceğini ilgili makaleleri incelememizdir. Biz de denemek için öncelikle sistemimizi bu şekilde geliştirdik.


<h3>3.1- LSTM (Long-Short Term Memory) Algoritması</h3>
    
Öncelikle bilindiği üzere LSTM derin öğrenme algoritmalarından en popüleri olan RNN(Recurrent Neural Network)'nin uzun süreli bağımlılıkları öğrenebildiği özel bir türüdür. 

<h3>RNN Nedir ?</h3>
    
Rnn'ler bir sonraki adımı tahmin etmek için kullanılan bir çeşit Derin Öğrenme Yapılarıdır. Diğer Derin Öğrenme Yapılarından en büyük farkları ise <b>hatırlamalarıdır.</b> Diğer bir farkı ise girdilerin birbirleri ile ilişkili olmasıdır. RNN’ler bir sonraki adımı takip edebilmek için girdiler arasında ilişkiler kurarlar ve eğitilirken tüm ilişkilerini hatırlarlar.

RNN’ler kurmuş oldukları ilişkilerin kalıcı olması için kendi içlerinde dönen döngü benzeri bir yapı kullanırlar. Tekrarlayan sinir ağları döngülere sahiptir. Aslında çıkan sonuç, bir sonrakini besliyor. Bu yapının açık hali;

<h4>RNN Yapısı :</h4>

<img src="https://i.hizliresim.com/3d7w94.png">

    
    
    

Uzun-Kısa Süreli Bellek Ağları, uzun süreli bağımlılıkları öğrenebilen özel bir RNN türüdür. Ayrıca uzun kısa süreli bellek derin öğrenme alanında kullanılan yapay bir tekrarlayan sinir ağı mimarisidir. Standart ileri beslemeli sinir ağlarının aksine, LSTM'nin geri bildirim bağlantıları vardır. Yalnızca tek veri noktalarını değil, aynı zamanda tüm veri dizilerini işleyebilir. 
LSTM lerin tasarlanma amacı uzun vadeli bağımlılık sorununu önlemektir. Varsayımsal olarak bilgiyi uzun süreler boyunca hatırlarlar.

Tüm tekrarlayan sinir ağları, sinir ağının tekrar eden modüllerinin bir zinciridir. 

Standart bir RNN’deki tekrarlayan modül tek bir katman içerir. LSTM’ler de bu zincir benzeri yapıya sahiptir, ancak tek bir nöral ağ katmanı yerine 4 tanesine sahiptir.

<b>Ayrıca LSTM'in bir katında 128 adet bellek hücresi bulunur.</b>


<h3>LSTM Yapısı</h3>

<img src="https://i.hizliresim.com/3vqfb3.JPG">

<b>Forget Gate (Unutma Kapısı):</b>
Hangi bilginin tutulacağı veya unutulacağına karar verir. Mantığı hiç karmaşık değil. Matematikte öğrendiğimiz gibi, bir sayı 0 ile çarpılırsa ne kadar büyük olursa olsun sonuç 0 olur. Burada da aynı mantıkla işlem yapılıyor. Unutmak için girdinin ağırlığına 0 verilir.
Bir önceki gizli katmandan gelen bilgiler ve güncel bilgiler Sigmoid Fonksiyonundan geçer. 0'a ne kadar yakınsa o kadar unutulacak, 1'e ne kadar yakınsa o kadar tutulacak demektir.

<b>Input Gate (Girdi Kapısı):</b>
Cell State’i güncellemek için kullanılır. Öncelikle Forget Gate’de (Unutma Kapısı) olduğu gibi Sigmoid fonksiyonu uygulanır, hangi bilginin tutulacağına karar verilir. Daha sonra ağı düzenlemek için Tanh fonksiyonu yardımıyla -1,1 arasına indirgenir ve çıkan iki sonuç çarpılır.

<b>Cell State:</b>
Cell State’in hücre içerisindeki en önemli görevi bilgiyi taşımaktır. Taşınması gereken verileri alır ve hücre sonuna, oradan da diğer hücrelere taşır. Yani ağ üzerinde veri akışını Cell State yardımıyla sağlarız. İlk olarak Forget Gate’den (Unutma Kapısı) gelen sonuç ile bir önceki katmanın sonucu çarpılır. Daha sonra Input Gate’den (Girdi Kapısı) gelen değer ile toplanır.

<b>Output Gate (Çıktı Kapısı):</b>
Bir sonraki katmana gönderilecek değere karar verir. Bu değer, tahmin için kullanılır. Öncelikle bir önceki değer ile şu anki girdi Sigmoid fonksiyonundan geçer. Cell State’den gelen değer Tanh fonksiyonundan geçtikten sonra iki değer çarpılır ve bir sonraki katmana “Bir önceki değer” olarak gider. Cell State ilerler.

<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/1.png">

<h3>LSTM Trainer Kısmı</h3>

In [ ]:
#pip3 install keras
#pip3 install tensorflow
#pip3 install nltk
#pip3 install gensim

from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.models import Sequential,load_model
from sklearn.model_selection import train_test_split

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import gensim

import nltk
from nltk.corpus import stopwords
import re

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import os

dataset = pd.read_csv('dataset.csv')
dataset.head()
dataset.sort_values("Body", inplace = True)
dataset = dataset.drop(columns="B")

dataset.drop_duplicates(subset ="Body",keep = False, inplace = True)
                     

def optimizasyon(dataset):
    dataset = dataset.dropna()

    stop_words = set(stopwords.words('turkish'))
    noktalamaIsaretleri = ['•', '!', '"', '#', '”', '“', '$', '%', '&', "'", '–', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '…']
    stop_words.update(noktalamaIsaretleri)

    for ind in dataset.index:
        body = dataset['Body'][ind]
        body = body.lower()
        body = re.sub(r'http\S+', '', body)
        body = re.sub('\[[^]]*\]', '', body)
        body = (" ").join([word for word in body.split() if not word in stop_words])
        body = "".join([char for char in body if not char in noktalamaIsaretleri])
        dataset['Body'][ind] = body
    return dataset

dataset = optimizasyon(dataset)


X = dataset.loc[:,"Body"]
y = dataset.loc[:,"Label"]

print(X)


print(y)

X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size = 0.2, random_state = 28) 

# Word2Vec yani dökümanı vektöre çevirecek gensim modeli için gerekli parametreler
maxmesafe = 2 #Bir cümle içindeki mevcut ve tahmin edilen kelime arasındaki maksimum mesafe
minfrekans = 1 #Toplam sıklığı bundan daha düşük olan kelimeleri göz ardı eder
vektor_boyut = 200 #Öznitelik vektörlerinin boyutluluğu
maxlen = 1000 #Bir  metninin maksimum uzunluğu


X_egitim_splited = [metin.split() for metin in X_egitim]
w2v_model = gensim.models.Word2Vec(sentences = X_egitim_splited, vector_size=vektor_boyut, window = maxmesafe,  min_count = minfrekans)                                          
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_egitim_splited)   #vektore donüştürme
X_egitim_tok = tokenizer.texts_to_sequences(X_egitim_splited)
kelime_index = tokenizer.word_index
kelime_sayi = len(kelime_index) + 1
X_egitim_tok_pad = pad_sequences(X_egitim_tok, maxlen=maxlen)  #butun x eğitim verilerini vektore donusturur


print('Sözlük boyutu: ', kelime_sayi) #toplam kac tane farklı kelime varsa bilgisini verir

matris = np.zeros((kelime_sayi, vektor_boyut))
for kelime, i in kelime_index.items():
    matris[i] = w2v_model.wv[kelime]

model = Sequential()
model.add(Embedding(matris.shape[0], 
                    output_dim=matris.shape[1],
                    weights=[matris], 
                    input_length=maxlen, 
                    trainable=False))
model.add(LSTM(units=32))   
model.add(Dense(1, activation='sigmoid'))   # Aktivasyon fonksiyonu olarak "Sigmoid" i seçiyoruz
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])    # Optimizer parametresi olarak "adam", loss için ise "binary_crossentropy" seçiyoruz.

model.summary()


model.fit(X_egitim_tok_pad, y_egitim, validation_split=0.2, epochs=30, batch_size = 64, verbose = 1)

model.save('egitilmis_model.h5')

print("Model eğitildi ve kayıt edildi !")

<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/2.png">



<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/3.png">



<h3>LSTM Prediction Kısmı</h3>


In [ ]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
maxlen = 1000

# Bir metini stopword kelimelerden, noktalama işaretlerinden, linklerden ve gereksiz yapılardan temizleyen optimizasyon fonksiyonu.
# Yapay zeka tarafından yorumlanacak veri ilk olarak bu aşamadan geçer
def optimizasyon(metin):
    stop_words = set(stopwords.words('turkish'))
    noktalamaIsaretleri = ['•', '!', '"', '#', '”', '“', '$', '%', '&', "'", '–', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '…']
    stop_words.update(noktalamaIsaretleri)
    metin = metin.lower()
    metin = re.sub(r'http\S+', '', metin)
    metin = re.sub('\[[^]]*\]', '', metin)
    metin = (" ").join([word for word in metin.split() if not word in stop_words])
    metin = "".join([char for char in metin if not char in noktalamaIsaretleri])
    return metin

# Tokenizer sınıfı kullanılarak düz metin, yapay zekanın yorumlayacağı hale getirilir.
def metinDonustur(testData,dataset):
    testData_splited = [testData.split() ]                              
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(testData_splited)
    testData_tok = tokenizer.texts_to_sequences(testData_splited)
    kelime_index = tokenizer.word_index
    kelime_sayi = len(kelime_index) + 1
    testData_tok_pad = pad_sequences(testData_tok, maxlen)
    text_test_tok = tokenizer.texts_to_sequences(dataset.loc[:,"Body"])
    text_test_tok_pad = pad_sequences(text_test_tok, maxlen=maxlen)
    print(text_test_tok_pad)
    return testData_tok_pad


def datasetOptimizasyon(dataset):
    dataset = dataset.dropna()
    stop_words = set(stopwords.words('turkish'))
    noktalamaIsaretleri = ['•', '!', '"', '#', '”', '“', '$', '%', '&', "'", '–', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '…']
    stop_words.update(noktalamaIsaretleri)

    for ind in dataset.index:
        body = dataset['Body'][ind]
        body = body.lower()
        body = re.sub(r'http\S+', '', body)
        body = re.sub('\[[^]]*\]', '', body)
        body = (" ").join([word for word in body.split() if not word in stop_words])
        body = "".join([char for char in body if not char in noktalamaIsaretleri])
        dataset['Body'][ind] = body
    return dataset

dataset = pd.read_csv('dataset.csv')

dataset = datasetOptimizasyon(dataset)

model = load_model("egitilmis_model.h5")


def tahminEt(inputData):

    inputData = optimizasyon(inputData)
    testData = metinDonustur(inputData,dataset)

    print("haber: \n" + inputData)
    pred = model.predict([testData]) 
    print(pred)
    if (pred > 0.98):
        print("Bu yorum insan üretimi olarak tahmin edilmiştir")
        return True
    else:
        print("Bu yorum makine üretimi (sahte) olarak tahmin edilmiştir.")
        return False


haber = input("Yorum girin")
tahminEt(haber)

<h3>LSTM Algoritması ile Sistemin Çalıştırılma Çıktıları</h3>


<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/4.jpg">



<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/5.png">



<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/6.png">



<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/8.png">


<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/9.png">

<h2>3.2- Lojistik Regrasyon Algoritması</h2>

<b>Lojistik regresyon,</b> bir sonucu belirleyen bir veya daha fazla bağımsız değişken bulunan bir veri kümesini analiz etmek için kullanılan istatistiksel bir yöntemdir. 

<b>Sonuç, ikili bir değişkenle ölçülür (yalnızca iki olası sonuç vardır). Yani 1 (Doğru, başarı), ya da 0 (Yanlış, hata) olarak kodlanmış verileri içerir.</b>

Amacı, iki yönlü karakteristiği (bağımlı değişken = yanıt veya sonuç değişkeni) ile ilgili bir dizi bağımsız (öngörücü veya açıklayıcı) değişken arasındaki ilişkiyi tanımlamak için en uygun (henüz biyolojik olarak makul) modeli bulmaktır. 

Lojistik regresyon, ilgi karakteristiklerinin varlığının olasılığını logit dönüşümünü tahmin etmek için bir formülün katsayılarını (ve standart hatalarını ve önem seviyelerini) üretir.

* Projemizde iki adet parametre olduğu için yani ya sahte yorum ya da makine yorumu diye karar verileceği için bu algoritmanın kullanılmasının uygun olacağına karar verdik

<h3>Lojistik Regrasyon Train Kısmı</h3>


In [ ]:
#pip3 install pickle-mixin
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle

#Veri setini oku
dataset = pd.read_csv('dataset.csv')
dataset.head()
dataset.sort_values("Body", inplace = True)
dataset = dataset.drop(columns="B")

#Aynı verileri sil
dataset.drop_duplicates(subset ="Body",keep = False, inplace = True)


#Optimizasyon Fonksiyonu
def optimizasyon(dataset):
    dataset = dataset.dropna()

    stop_words = set(stopwords.words('turkish'))
    noktalamaIsaretleri = ['•', '!', '"', '#', '”', '“', '$', '%', '&', "'", '–', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '…']
    stop_words.update(noktalamaIsaretleri)

    for ind in dataset.index:
        body = dataset['Body'][ind]
        body = body.lower()
        body = re.sub(r'http\S+', '', body)
        body = re.sub('\[[^]]*\]', '', body)
        body = (" ").join([word for word in body.split() if not word in stop_words])
        body = "".join([char for char in body if not char in noktalamaIsaretleri])
        dataset['Body'][ind] = body
    return dataset


dataset = optimizasyon(dataset)


yorumlar_makina = dataset[dataset['Label']==0]
yorumlar_insan = dataset[dataset['Label']==1]

tfIdf = TfidfVectorizer( binary=False, ngram_range=(1,3))


makina_cv = tfIdf.fit_transform(yorumlar_makina['Body'].tolist())
insan_cv = tfIdf.fit_transform(yorumlar_insan['Body'].tolist())



X = dataset['Body']
y = dataset['Label']

x_tv = tfIdf.fit_transform(X)

x_train_tv, x_test_tv, y_train_tv, y_test_tv = train_test_split(x_tv, y, test_size=0.2, random_state=0)


log_tv = LogisticRegression() 
log_tv.fit(x_train_tv,y_train_tv)

pickle.dump(log_tv, open("egitilmis_model", 'wb'))
print("Lojistik Regresyon modeli eğitildi ve kayıt edildi !")


pickle.dump(tfIdf, open("vektorlestirici", 'wb'))
print("Tf-Idf vektörleştirici modeli kayıt edildi !")



<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/1-2.png">

<h3>Lojistik Regresyon Prediction Kısmı</h3>

In [ ]:
import pickle
from sklearn.linear_model import LogisticRegression
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer




def optimizasyon(metin):

    stop_words = set(stopwords.words('turkish'))
    noktalamaIsaretleri = ['•', '!', '"', '#', '”', '“', '$', '%', '&', "'", '–', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '…']
    stop_words.update(noktalamaIsaretleri)

    body = metin
    body = body.lower()
    body = re.sub(r'http\S+', '', body)
    body = re.sub('\[[^]]*\]', '', body)
    body = (" ").join([word for word in body.split() if not word in stop_words])
    body = "".join([char for char in body if not char in noktalamaIsaretleri])
    return body


LogisticRegressionModel = pickle.load(open("egitilmis_model", 'rb'))

tfIdf = pickle.load(open("vektorlestirici", 'rb'))



def tahminEt(testData):
    testData = optimizasyon(testData)
    vektorlestirilmis_test_verisi = tfIdf.transform([testData])


    print(vektorlestirilmis_test_verisi)
    #4 yıldız çünkü jelatini açılmıştı jelatininin açılması dışında her şey yolundaydı
    tahminSonuc = LogisticRegressionModel.predict(vektorlestirilmis_test_verisi)

    print(tahminSonuc)



inp = input("Yorum giriniz")


tahminEt(inp)

<h3>Lojisik Regrasyonun Doğruluğunu Ölçme</h3>

In [ ]:
#pip3 install pickle-mixin
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle

#nltk.download("punkt")


dataset = pd.read_csv('dataset.csv')
dataset.head()
dataset.sort_values("Body", inplace = True)
dataset = dataset.drop(columns=["B","id"])

dataset.drop_duplicates(subset ="Body",keep = False, inplace = True)

def optimizasyon(dataset):
    dataset = dataset.dropna()

    stop_words = set(stopwords.words('turkish'))
    noktalamaIsaretleri = ['•', '!', '"', '#', '”', '“', '$', '%', '&', "'", '–', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '…']
    stop_words.update(noktalamaIsaretleri)

    for ind in dataset.index:
        body = dataset['Body'][ind]
        body = body.lower()
        body = re.sub(r'http\S+', '', body)
        body = re.sub('\[[^]]*\]', '', body)
        body = (" ").join([word for word in body.split() if not word in stop_words])
        body = "".join([char for char in body if not char in noktalamaIsaretleri])
        dataset['Body'][ind] = body
    return dataset


dataset = optimizasyon(dataset)


pat1 = '@[^ ]+'
pat2 = 'http[^ ]+'
pat3 = 'www.[^ ]+'
pat4 = '#[^ ]+'
pat5 = '[0-9]'

combined_pat = '|'.join((pat1, pat2, pat3, pat4, pat5))


for ind in dataset.index:
    t = dataset['Body'][ind]
    t = t.lower()
    stripped = re.sub(combined_pat, '', t)
    tokens = word_tokenize(stripped)
    words = [x for x  in tokens if len(x) > 1]
    sentences = " ".join(words)
    dataset['Body'][ind] = sentences

x = dataset['Body']
y = dataset['Label']


tv = TfidfVectorizer(stop_words='turkish', binary=False, ngram_range=(1,3))
x_tv = tv.fit_transform(x)
x_train_tv, x_test_tv, y_train_tv, y_test_tv = train_test_split(x_tv, y, test_size=0.2, random_state=0)

log_tv = LogisticRegression() 
log_tv.fit(x_train_tv,y_train_tv)

y_pred_tv = log_tv.predict(x_test_tv)
print(confusion_matrix(y_test_tv,y_pred_tv))
print(classification_report(y_test_tv,y_pred_tv))

<h3>Lojistik Regrasyon Algoritması ile Sistemin Çalıştırılma Çıktıları</h3>

<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/2-2.png">

<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/3-2.png">

<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/4-1.png">

<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/5-2.png">



<img src="https://artemiseticaret.com/wp-content/uploads/2021/05/6-2.png">

<h2>4- Algoritmaların Karşılaştırılması (Sonuç)</h2>

<blockquote cite="https://www.huxley.net/bnw/four.html">Sonuç olarak Lojistik Regrasyon algoritması için geliştirilen sistemin doğruluk oranı daha yüksektir. Lstm kullanarak sistemi eğitirken yaklaşık 15 dakikada model geliştirilmiştir. Lojistik regrasyon daha basit yapıda olduğu için 5 saniyede model oluşturulmuştur. Böyle bir proje gerçekleştirilirken maliyet açısından değerlendirilecek olursa <b>Lojistik regrasyonu</b> tercih edilmelidir. Ayrıca bu proje 0 ya da 1 gibi değerlere yakın olduğu için Lojistik Regrasyon daha uygun bir algoritmadır.</blockquote>